In [ ]:
import os
import time
import hmac
import base64
import hashlib

import httpx
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

In [ ]:
class Signature:

    @staticmethod
    def generate(timestamp, method, uri, secret_key):
        message = "{}.{}.{}".format(timestamp, method, uri)
        hash = hmac.new(bytes(secret_key, "utf-8"), bytes(message, "utf-8"), hashlib.sha256)

        hash.hexdigest()
        return base64.b64encode(hash.digest())

In [ ]:
def get_headers():
	x_api_key = os.getenv("AD_NAVER_API_KEY")
	x_customer = os.getenv("AD_NAVER_CUSTOMER_ID")
	x_timestamp = str(round(time.time() * 1000))
 
	secret_key = os.getenv("AD_NAVER_SECRET_KEY")
	method = "GET"
	uri = "/keywordstool"

	# X-Signature
	# : X-Timestamp, method, uri 값을 점(.)으로 연결한 뒤 HmacSHA256 알고리즘으로 암호화한 후 Base64로 인코딩한 값.
	sign = f"{x_timestamp}.{method}.{uri}"
	signature_encrypted = hmac.new(
	    secret_key.encode(), sign.encode(), hashlib.sha256
	).digest()
	x_signature = base64.b64encode(signature_encrypted)

	headers = {
        "Content-Type": "application/json; charset=UTF-8",
	    "X-API-KEY": x_api_key,
	    "X-CUSTOMER": x_customer,
	    "X-Timestamp": x_timestamp,
	    "X-Signature": x_signature,
	}

	return headers

In [ ]:
headers = get_headers()

url = "https://api.searchad.naver.com/keywordstool"
params = {
    "hintKeywords": "효성에프엠에스,네이버페이,샘올트먼",
    "showDetail": 0
}

resp = httpx.get(url=url, params=params, headers=headers)

In [ ]:
resp.json()

In [ ]:
df = pd.DataFrame(resp.json()['keywordList'])
df

In [ ]:
df["monthlyMobileQcCnt"].sum()

In [ ]:
df.loc[df["monthlyMobileQcCnt"] == "< 10", "monthlyMobileQcCnt"] = 0
df.loc[df["monthlyPcQcCnt"] == "< 10", "monthlyPcQcCnt"] = 0
df

In [ ]:
df.rename(
    columns={
        "relKeyword": "keyword",
        "monthlyPcQcCnt": "monthly_qc_pc_cnt",
        "monthlyMobileQcCnt": "monthly_qc_mobile_cnt",
    },
    inplace=True,
)
df